# Set Up The Environment

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import re
import os
import time
from ckiptagger import WS, POS, NER, construct_dictionary # tokenization

# Setup work directory
os.chdir('/Users/deankuo/Desktop/python/dissertation/Taiwan-Manifesto-Analysis-A-ML-Approach')
os.listdir()

# Import utils.py
from Utils.utils_token import (
    flatten,
    text_select,
    load_text,
    split_content,
    
)
matplotlib.rcParams['font.family'] = 'Times New Roman'

# CKIP module
CKIP_PATH = "../CKIP_TAGGER"
ws = WS(CKIP_PATH) # 斷詞
pos = POS(CKIP_PATH) # 詞性標註
ner = NER(CKIP_PATH) # 命名實體識別

2024-02-18 15:40:22.826353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/deankuo/opt/anaconda3/lib/python3.8/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
2024-02-18 15:40:35.869551: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with t

In [2]:
# 加入自定義字典
word_to_weight = {'823': 1, 'ECFA': 1, '2300': 1, '台26線': 1, '台74線': 1, '12年國教': 1, 'BOT': 1, '88快速道路': 1, '台27線': 1, '台61線': 1, '十二年國教': 1, '國道10號': 1,
                  '台88號': 1, 'M型': 1, '205兵工廠': 1, '北二高': 1, '台65線': 1, 'CEPA': 1, 'FTA': 1, '科學園區': 1, '228': 1, 'MIT': 1, '202兵工廠': 1, '86快速道路': 1, '國道8號': 1,
                  '台64': 1, '台66': 1, 'iBike': 1, 'MRT': 1, 'TPP': 1, 'TIFA': 1, 'TPP':1, '台22': 1, '台29': 1, '國10': 1, '國1': 1, '318': 1, 'NCC':1, 'PM2.5': 1, 'YouBike': 1, 
                  '台68': 1, '快速道路': 1, 'NGO': 1, 'NPO': 1, 'U-Bike': 1, 'LGBTQ': 1, '三七五減租': 1, '小三通': 1, '大三通': 1, '基礎建設': 1, '戒急用忍': 1, '社會役': 1, '非核家園': 1,
                  '教育券': 1, '九二共識': 1}
dictionary = construct_dictionary(word_to_weight)

# 停用詞
with open("./Data/stopwords_zh-tw.txt", encoding="utf-8") as fin:
    stopwords = fin.read().split("\n")[1:]

In [42]:
years = [1992, 1995, 1998, 2001, 2004, 2008, 2012, 2016, 2020, 2024]
dfs = {}
for year in years:
    dfs[year] = pd.read_excel(f"./Data/Dataset/{year}.xlsx")

In [ ]:
for k, v in dfs.items():
    dfs[k] = split_content(dfs[k])

In [46]:
sum = 0
for k, v in dfs.items():
    sum += dfs[k].shape[0]

print(sum)

40575


# Tokenization

In [22]:
# Test the text selection function
text = dfs[2024].loc[0, 'CONTENT']

sentence_, pos_, ner_, token_ = load_text(text, test=1)
print()
print('=====')
print("原文：")
print(text)
print("斷詞後：")
print(sentence_)
print("斷詞後+詞性標注：")
print(pos_)
print("透過詞性篩選的斷詞: ")
print(token_)
print("命名實體辨識: ")
print(ner_)
print('=====')


=====
原文：
海口新選擇、改變有新希望：
●強力監督政府，拒當「黨意」立委。
●反對巧立名目編列「特別預算」，拒絕債留子孫。
●揭示《ＣＯＶＩＤ１９疫苗採購公文》、進口蛋採購合約。
教育平等、撐住每個孩子：
●國中小及中等教育教育預算提高至ＧＤＰ２.５％以上（＃２０１９年臺灣２％、日本２.６％、韓國３.４％、ＯＥＣＤ各國３.４％。）。
●偏鄉、非山非市學校建立智慧校園，實現平等教育。
●成立三級《網路成癮對策中心》。
顧老護幼、實現世代正義：

●推動智慧遠距醫療、智慧遠距照顧，弭平城鄉差距。
●青年創業、育成輔導獎勵。
●全面檢討修訂貨物稅、所得稅、房屋稅。
●６５歲（含）以上長輩健保費由中央補助。
●一鄉鎮一大型共融式全齡公園。
產業創新、地方創生：

●全面修繕縣內漁港，轉型漁業觀光港口，漁業升級智慧產業。
●依物價波動調升公糧收購價格，調整天然災害救助金額。
●打造生態、文化觀光軸帶，地方創生、農村再生、產業創新拚經濟。
斷詞後：
海口 選擇 改變 希望 強力 監督 政府 拒當 黨意 立委 反對 巧立 名目 編列 特別 預算 拒絕 子孫。
 揭示 ＣＯＶＩＤ １９ 疫苗 採購 公文 進口 採購 合約 教育 平等 撐住 孩子 國中小 中等 教育 教育 預算 提高 ＧＤＰ ２.５％ 以上 ２０１９年 臺灣 ２％ 日本 ２.６％ 韓國 ３.４％ ＯＥＣＤ ３.４％ 。
 偏鄉 非山非市 學校 建立 智慧 校園 實現 平等 教育 成立 網路 成癮 對策 中心 護幼 實現 世代 正義 

 推動 智慧 遠距 醫療 智慧 遠距 照顧 弭平 城鄉 差距 。
 青年 創業 育成 輔導 獎勵 。
 全面 檢討 修訂 貨物稅 所得稅 房屋稅 。
 ６５ 以上 長輩 健保費 中央 補助 。
 鄉鎮 大型 共融式 全齡 公園 。
 產業 創新 地方 創生 

 全面 修繕 漁港 轉型 漁業 觀光 港口 漁業 升級 智慧 產業 物價 波動 調升 公糧 收購 價格 調整 天然 災害 救助 金額 打造 生態 文化 觀光 軸帶 地方 創生 農村 再生 產業 創新 經濟
斷詞後+詞性標注：
['海口(Nc)', '選擇(VC)', '改變(VC)', '希望(Na)', '強力(D)', '監督(VC)', '政府(Na)', '拒當(VG)', '黨意(Na)', '立委(Na)'

## Iterate Each Year Data

In [ ]:
def tokenization(year: int, df: pd.DataFrame) -> pd.DataFrame:
    start = time.time()
    df[['SENTENCE', 'TOKEN']] = df['CONTENT'].apply(load_text).apply(pd.Series)
    end = time.time()
    # 每一年份使用CPU的運算時間約為10分鐘
    print(f"{year}年選舉公報的斷詞運算時間為: {round((end - start) / 60, 2)} 分")
    return df

for key, value in dfs.items():
    dfs[key] = tokenization(key[3:], value)